In [51]:
import pandas as pd
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0

In [52]:
def get_conflict_training_data(filepath_tsv, conflict_events=True):
    df = pd.read_csv(filepath_tsv, sep='\t')
    df['spanish'] = df.iloc[:,0].apply(lambda x: True if detect(x) == 'es' else False)
    df = df[df.spanish]
    if conflict_events:
        df = df[df.iloc[:,1:23].any(axis=1)]
        df['conflict'] = 1
    else:
        df = df[~df.iloc[:,1:23].any(axis=1)]
        df['conflict'] = 0
    
    df = df.drop(columns=['spanish'])
    df.drop(df.columns[list(range(1,39))],axis=1,inplace=True)

    df.columns.values[0] = 'article'
    return df


In [53]:
propagated_df = get_conflict_training_data("annotated_data_for_filter_training/propagated_InsightCrime.tsv")  
annotated_df = get_conflict_training_data("annotated_data_for_filter_training/annotated_InsightCrime.tsv")

In [65]:
df_complete = pd.concat([propagated_df, annotated_df], ignore_index=True)
df_complete.to_csv("training_data.csv")

In [56]:
len(df_complete)

6094

In [57]:
propagated_df_negative = get_conflict_training_data("annotated_data_for_filter_training/propagated_InsightCrime.tsv", False)
annotated_df_negative = get_conflict_training_data("annotated_data_for_filter_training/annotated_InsightCrime.tsv", False)


In [58]:
df_negative_complete = pd.concat([propagated_df_negative, annotated_df_negative], ignore_index=True)
df_negative_complete.head()


,article,conflict
0,incubadoras del crimen organizado en Latinoamé...,0
1,Los hombres que vendían a las mujeres. Pregunt...,0
2,El alcalde Tres Kiebres de Guatemala. Un ambie...,0
3,Élites y crimen organizado: Marco conceptual. ...,0
4,Élites y crimen organizado. El crimen organiza...,0


In [59]:
df_negative_complete.to_csv("negative_training_data.csv")

In [60]:
res = [df_complete, df_negative_complete]
df_total = pd.concat(res)
df_total.to_csv("final_training_data_v2.csv")

In [61]:
df_total = pd.read_csv("final_training_data_v2.csv", index_col=0)